# Convert-search

Invoke a Common Workflow Language CommandLineTool bash script to invoke `yq` to:
- extract the discovered STAC Items `self` link `href` that is used as an input in the water bodies detection sub-workflow
- extract the aoi bounding box from the `search_request` input parameter

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/cwl-cli/convert-search/



## Setup

In [ ]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

curl -q -L https://github.com/eoap/zarr-cloud-native-format/releases/download/0.5.0/app-water-bodies.0.5.0.cwl > ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl 2> /dev/null

## Run the convert-seach step

Inspect and use `cwltool` to run the CommandLineTool definition:


In [ ]:
cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][1]' -

The bash script to run is:

In [ ]:
yq e '.["$graph"][1].requirements[]
      | select(.class == "InitialWorkDirRequirement")
      | .listing[0].entry' \
  "${WORKSPACE}/cwl-workflow/app-water-bodies.cwl"

Run the CWL description:

In [ ]:
cat <<EOF > convert-search-params.yaml
search_request:
  bbox:
  - -121.399
  - 39.834
  - -120.74
  - 40.472
  collections:
  - sentinel-2-l2a
  datetime_interval:
    end:
      value: '2021-08-01T23:59:59'
    start:
      value: '2021-06-01T00:00:00'
  limit: 20
  max-items: 10

search_results:
  class: File
  path: "${WORKSPACE}/runs/discovery-output.json"
EOF

cat convert-search-params.yaml | yq .

In [ ]:


cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl#convert-search \
    convert-search-params.yaml > convert-search-results.json 2> convert-search.log

Let's look at the content of the stderr:

In [ ]:
cat convert-search.log | egrep -v "WARNING|JSHINT"

Let's inspect the stdout produced. There are two `output` blocks with the id `aoi` and `items`. 

These are what the water bodies detection sub-workflow expects as inputs

In [ ]:
cat convert-search-results.json | jq . -